# Check Capse.jl
In this notebook we are going to check the accuracy of our emulators.

In [ ]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()
Pkg.resolve()

In [ ]:
using Statistics, DataFrames, JSON3, Plots, LinearAlgebra, BenchmarkTools, NPZ, Random
using Base: @kwdef
using BSON: @save
using BSON: @load
using BSON
using LaTeXStrings
using SimpleChains
using Static
using NPZ
using Capse

In [ ]:
path_test = "../data/Capse_Extended_20000/aacwdoenqixtxi/"
json_string_test = read(path_test*"cosmology_capse.json", String)
cosmo_pars_test = JSON3.read(json_string_test)
CℓTT_test = npzread(path_test*"clTT.npy")[3:5001]
CℓEE_test = npzread(path_test*"clEE.npy")[3:5001]
CℓTE_test = npzread(path_test*"clTE.npy")[3:5001]
CℓPP_test = npzread(path_test*"clPP.npy")[3:5001]
ℓ = npzread(path_test*"ll.npy")[3:5001];

In [ ]:
ll_SO = npzread("../data/noise/noise_SO_ll.npy")
ll_SO = ll_SO[3:4999]
fac = ll_SO .* (ll_SO .+ 1) ./ (2π)
TT_n_SO = npzread("../data/noise/noise_SO_TT.npy")[3:4999] .* fac
EE_n_SO = npzread("../data/noise/noise_SO_EE.npy")[3:4999] .* fac
PP_n_SO = npzread("../data/noise/noise_SO_PP.npy")[3:4999];

In [ ]:
ll_S4 = npzread("../data/noise/noise_S4_ll.npy")[3:4999]
fac = ll_S4 .* (ll_S4 .+ 1) ./ (2π)
fac_kk = (ll_S4 .* (ll_S4 .* 1)).^2 / 4
TT_n_S4 = npzread("../data/noise/noise_S4_TT.npy")[3:4999] .* fac
EE_n_S4 = npzread("../data/noise/noise_S4_EE.npy")[3:4999] .* fac
PP_n_S4 = npzread("../data/noise/noise_S4_PP.npy")[3:4999];

In [ ]:
plot_font = "Computer Modern"

Plots.default(titlefont = (16, plot_font), fontfamily=plot_font,
        linewidth=2, framestyle=:box, fg_legend =:black, label=nothing, grid=false, tickfontsize=12, size = (550, 400), labelfontsize = 13, dpi = 200, minorgrid = true, xticks = [1,10,100, 1000], xscale=:log10)

plt = plot(ll_SO, TT_n_SO)
plot!(plt, ll_S4, TT_n_S4)

In [ ]:
plot_font = "Computer Modern"

Plots.default(titlefont = (16, plot_font), fontfamily=plot_font,
        linewidth=2, framestyle=:box, fg_legend =:black, label=nothing, grid=false, tickfontsize=12, size = (550, 400), labelfontsize = 13, dpi = 200, minorgrid = true, xticks = [1,10,100, 1000], xscale=:log10)

plt = plot(ll_SO, EE_n_SO)
plot!(plt, ll_S4, EE_n_S4)

In [ ]:
plot(PP_n_SO, xscale=:log10, yscale=:log10)
plot!(CℓPP_test)
plot!(PP_n_S4)
#plot!(PP_n_S4)

In [ ]:
input_test = [cosmo_pars_test["ln10As"], cosmo_pars_test["ns"], cosmo_pars_test["H0"], cosmo_pars_test["ombh2"], cosmo_pars_test["omch2"], cosmo_pars_test["tau"]];

In [ ]:
mlpd = SimpleChain(
  static(6),
  TurboDense(tanh, 64),
  TurboDense(tanh, 64),
  TurboDense(tanh, 64),
  TurboDense(tanh, 64),
  TurboDense(tanh, 64),
  TurboDense(identity, 4999)
);

# $C^{TT}(\ell)$ emulator test

In [ ]:
weights_folder = "../data/weights/weights_cosmopowerspace_10000/"
ℓ = npzread(weights_folder*"l.npy")

weights_TT = npzread(weights_folder*"weights_TT_lcdm.npy")
trained_emu_TT = Capse.SimpleChainsEmulator(Architecture= mlpd, Weights = weights_TT)
CℓTT_emu = Capse.CℓEmulator(TrainedEmulator = trained_emu_TT, ℓgrid = ℓ,
                             InMinMax = npzread(weights_folder*"inMinMax_lcdm.npy"),
                             OutMinMax = npzread(weights_folder*"outMinMaxCℓTT_lcdm.npy"));

In [ ]:
CℓTT_capse = Capse.get_Cℓ(input_test, CℓTT_emu);

In [ ]:
@benchmark Capse.get_Cℓ($input_test, $CℓTT_emu)

In [ ]:
plt = Plots.plot(xlabel=L"ℓ", ylabel=L"\frac{\ell(\ell+1)}{2\pi}C^{TT}(\ell)",legend=:bottomright)
Plots.plot!(plt, ℓ, CℓTT_test)
Plots.plot!(plt, ℓ, CℓTT_capse)# .* ℓ .*(ℓ .+1))
plt

$$
\sigma_{\ell, \mathrm{CMB}}^{\{\mathrm{TT}, \mathrm{EE}\}}=\sqrt{\frac{2}{f_{\mathrm{sky}}(2 \ell+1)}}\left(C_{\ell, \text { true }}^{\{\mathrm{TT}, \mathrm{EE}\}}+N_{\ell}^{\{\mathrm{TT}, \mathrm{EE}, \phi \phi\}}\right)
$$
with $f_\mathrm{sky} = 1$ and $N_\ell=0$

In [ ]:
function compute_σXX(CℓXX, ℓ)
    result = Float64.(similar(ℓ))
    for i in 1:length(ℓ)
        result[i] = sqrt(2/( (2. *ℓ[i]+1.))) * (CℓXX[i])
    end
    return result
end

function compute_σXX(CℓXX, ℓ, NXX, f_sky)
    result = Float64.(similar(ℓ))
    for i in 1:length(ℓ)
        result[i] = sqrt(2/(f_sky .* (2. *ℓ[i]+1.))) * (CℓXX[i] + NXX[i])
    end
    return result
end

In [ ]:
σTT = compute_σXX(CℓTT_test, ℓ[3:5001]);
σTT_n = compute_σXX(CℓTT_test[1:4997], ℓ[3:4999], TT_n_SO, 1. );

## Deep test

In [ ]:
Cℓ_directory = "../data/HiCapse_extended_10000/"
Cℓ_directory = "../data/Capse_Extended_20000/"
function evaluate_residuals_XX(Directory,  CℓXX_emu, XX::String)
    my_values = zeros(20000, 4999)
    i = 0
    for (root, dirs, files) in walkdir(Directory)
        for file in files
            file_extension = file[findlast(isequal('.'),file):end]
            if file_extension == ".json"
                res = read_Cℓ_compute_res_XX(root, CℓXX_emu, XX)
                i += 1
                my_values[i, :] = res
            end
        end
    end
    return my_values
end


function read_Cℓ_compute_res_XX(location, CℓXX_emu, XX::String)
    json_string = read(location*"/cosmology_capse.json", String)
    cosmo_pars_test = JSON3.read(json_string)
    
    input_test = [cosmo_pars_test["ln10As"],
        cosmo_pars_test["ns"],
        cosmo_pars_test["H0"],
        cosmo_pars_test["ombh2"],
        cosmo_pars_test["omch2"],
        cosmo_pars_test["tau"]]

    ℓ = npzread(location*"/ll.npy")[3:5001]
    fac = ℓ .* (ℓ .*1) ./ (2π)
    
    CℓXX_test = (npzread(location*"/cl"*XX*".npy")[3:5001]) ./ fac
    
    CℓXX_capse = Capse.get_Cℓ(input_test, CℓXX_emu) ./ fac
    σXX = compute_σXX(CℓXX_test, ℓ)
    
    res = abs.(CℓXX_test .- CℓXX_capse)./σXX
        
    return res
end

function evaluate_residuals_XX(Directory, CℓXX_emu, XX::String, N_XX)
    my_values = zeros(20000, 4997)
    i = 0
    for (root, dirs, files) in walkdir(Directory)
        for file in files
            file_extension = file[findlast(isequal('.'),file):end]
            if file_extension == ".json"
                res = read_Cℓ_compute_res_XX(root, CℓXX_emu, XX, N_XX)
                i += 1
                my_values[i, :] = res
            end
        end
    end
    return my_values
end


function read_Cℓ_compute_res_XX(location, CℓXX_emu, XX::String, N_XX)
    json_string = read(location*"/cosmology_capse.json", String)
    cosmo_pars_test = JSON3.read(json_string)
    
    input_test = [cosmo_pars_test["ln10As"],
        cosmo_pars_test["ns"],
        cosmo_pars_test["H0"],
        cosmo_pars_test["ombh2"],
        cosmo_pars_test["omch2"],
        cosmo_pars_test["tau"]]

    ℓ = npzread(location*"/ll.npy")[3:4999]
    fac = ℓ .* (ℓ .*1) ./ (2π)
    
    CℓXX_test = (npzread(location*"/cl"*XX*".npy")[3:4999])# ./ fac
    
    CℓXX_capse = Capse.get_Cℓ(input_test, CℓXX_emu)[1:4997]# ./ fac
    σXX = compute_σXX(CℓXX_test, ℓ, N_XX, 0.4)
    
    res = abs.(CℓXX_test .- CℓXX_capse)./σXX
        
    return res
end

In [ ]:
function sort_residuals(residuals, n_output, n_elements)
    sorted_residuals = zeros(n_elements, n_output)
    for i in 1:n_output
        sorted_residuals[:,i] = sort(residuals[:,i])
    end
    final_residuals = zeros(3, n_output)

    final_residuals[1,:] = sorted_residuals[trunc(Int, (20000*0.68)),:]
    final_residuals[2,:] = sorted_residuals[trunc(Int, (20000*0.95)),:]
    final_residuals[3,:] = sorted_residuals[trunc(Int, (20000*0.99)),:]

    return final_residuals
end

In [ ]:
function evaluate_sorted_residuals_XX(Directory, CℓXX_emu, XX::String, N_XX)
    residuals = evaluate_residuals_XX(Directory, CℓXX_emu, XX::String, N_XX)
    n_elements, n_output = size(residuals)
    return sort_residuals(residuals, n_output, n_elements)
end

function evaluate_sorted_residuals_XX(Directory, CℓXX_emu, XX::String)
    residuals = evaluate_residuals_XX(Directory, CℓXX_emu, XX::String)
    n_elements, n_output = size(residuals)
    return sort_residuals(residuals, n_output, n_elements)
end

In [ ]:
sorted_residuals_TT = evaluate_sorted_residuals_XX(Cℓ_directory, CℓTT_emu, "TT")
sorted_residuals_TT_SO = evaluate_sorted_residuals_XX(Cℓ_directory, CℓTT_emu, "TT", TT_n_SO)
sorted_residuals_TT_S4 = evaluate_sorted_residuals_XX(Cℓ_directory, CℓTT_emu, "TT", TT_n_S4);

In [ ]:
npzwrite("sorted_residuals_TT.npy", sorted_residuals_TT)
npzwrite("sorted_residuals_TT_SO.npy", sorted_residuals_TT_SO)
npzwrite("sorted_residuals_TT_S4.npy", sorted_residuals_TT_S4)

In [ ]:
plt = Plots.plot(sorted_residuals_TT[1,:], xlabel=L"ℓ", ylabel=L"\frac{\left|\Delta C^{TT}\right|}{\sigma_{{TT}}}(\ell)\quad \mathrm{distribution}",legend=:topleft, label = L"68\%")
Plots.plot!(plt, sorted_residuals_TT[2,:], label = L"95\%")
Plots.plot!(plt, sorted_residuals_TT[3,:], label = L"99\%")
savefig("error_distribution_TT_CV.png")
savefig("error_distribution_TT_CV.pdf")
plt

In [ ]:
plt = Plots.plot(sorted_residuals_TT_SO[1,:], xlabel=L"ℓ", ylabel=L"\frac{\left|\Delta C^{TT}\right|}{\sigma_{{TT}}}(\ell)\quad \mathrm{distribution}",legend=:topleft, label = L"68\%")
Plots.plot!(plt, sorted_residuals_TT_SO[2,:], label = L"95\%")
Plots.plot!(plt, sorted_residuals_TT_SO[3,:], label = L"99\%", xscale=:log10)
savefig("error_distribution_TT_SO.png")
savefig("error_distribution_TT_SO.pdf")
plt

In [ ]:
plt = Plots.plot(sorted_residuals_TT_S4[1,:], xlabel=L"ℓ", ylabel=L"\frac{\left|\Delta C^{TT}\right|}{\sigma_{{TT}}}(\ell)\quad \mathrm{distribution}",legend=:topleft, label = L"68\%")
Plots.plot!(plt, sorted_residuals_TT_S4[2,:], label = L"95\%")
Plots.plot!(plt, sorted_residuals_TT_S4[3,:], label = L"99\%", xscale=:log10)
savefig("error_distribution_TT_S4.png")
savefig("error_distribution_TT_S4.pdf")
plt

# $C^{EE}(\ell)$ emulator test

In [ ]:
weights_EE = npzread(weights_folder*"weights_EE_lcdm.npy")
trained_emu_EE = Capse.SimpleChainsEmulator(Architecture= mlpd, Weights = weights_EE)
CℓEE_emu = Capse.CℓEmulator(TrainedEmulator = trained_emu_EE, ℓgrid = ℓ,
                             InMinMax = npzread(weights_folder*"inMinMax_lcdm.npy"),
                             OutMinMax = npzread(weights_folder*"outMinMaxCℓEE_lcdm.npy"));

## Quick test

In [ ]:
CℓEE_capse = Capse.get_Cℓ(input_test, CℓEE_emu);

In [ ]:
plot_font = "Computer Modern"

Plots.default(titlefont = (16, plot_font), fontfamily=plot_font,
        linewidth=2, framestyle=:box, fg_legend =:black, label=nothing, grid=false, tickfontsize=12, size = (550, 400), labelfontsize = 13, dpi = 200)

plt = Plots.plot(xlabel=L"ℓ", ylabel=L"\frac{\ell(\ell+1)}{2\pi}C^{EE}(\ell)",legend=:bottomright)
Plots.plot!(plt, ℓ, CℓEE_test)# .* ℓ .*(ℓ .+1))
Plots.plot!(plt, ℓ, CℓEE_capse)# .* ℓ .*(ℓ .+1))

In [ ]:
σEE = compute_σXX(CℓEE_test[1:4999], ℓ[3:5001]);

In [ ]:
plt = Plots.plot(xlabel=L"ℓ", ylabel=L"\frac{\left|\Delta C^{EE}\right|}{\sigma_{EE}}(\ell)",legend=:bottomright)
Plots.plot!(plt, ℓ[3:5001], abs.(CℓEE_test .- CℓEE_capse)./σEE )
savefig("residuals_ClEE.png")
plt

## Deep Test

In [ ]:
sorted_residuals_EE = evaluate_sorted_residuals_XX(Cℓ_directory, CℓEE_emu, "EE")
sorted_residuals_EE_SO = evaluate_sorted_residuals_XX(Cℓ_directory, CℓEE_emu, "EE", EE_n_SO)
sorted_residuals_EE_S4 = evaluate_sorted_residuals_XX(Cℓ_directory, CℓEE_emu, "EE", EE_n_S4);

In [ ]:
npzwrite("sorted_residuals_EE.npy",    sorted_residuals_EE)
npzwrite("sorted_residuals_EE_SO.npy", sorted_residuals_EE_SO)
npzwrite("sorted_residuals_EE_S4.npy", sorted_residuals_EE_S4)

In [ ]:
plt = Plots.plot(sorted_residuals_EE[1,:], xlabel=L"ℓ", ylabel=L"\frac{\left|\Delta C^{EE}\right|}{\sigma_{EE}}(\ell)\quad \mathrm{distribution}",legend=:topright, label = L"68\%")
Plots.plot!(plt, sorted_residuals_EE[2,:], label = L"95\%")
Plots.plot!(plt, sorted_residuals_EE[3,:], label = L"99\%")
savefig("error_distribution_EE_CV.png")
savefig("error_distribution_EE_CV.pdf")
plt

In [ ]:
plt = Plots.plot(sorted_residuals_EE[1,:], xlabel=L"ℓ", ylabel=L"\frac{\left|\Delta C^{EE}\right|}{\sigma_{EE}}(\ell)\quad \mathrm{distribution}",legend=:topright, label = L"68\%")
Plots.plot!(plt, sorted_residuals_EE[2,:], label = L"95\%")
Plots.plot!(plt, sorted_residuals_EE[3,:], label = L"99\%", ylimit = [0,1])
savefig("error_distribution_EE_CV_zoom.png")
savefig("error_distribution_EE_CV_zoom.pdf")
plt

In [ ]:
plt = Plots.plot(sorted_residuals_EE_SO[1,:], xlabel=L"ℓ", ylabel=L"\frac{\left|\Delta C^{EE}\right|}{\sigma_{EE}}(\ell)\quad \mathrm{distribution}",legend=:topright, label = L"68\%")
Plots.plot!(plt, sorted_residuals_EE_SO[2,:], label = L"95\%")
Plots.plot!(plt, sorted_residuals_EE_SO[3,:], label = L"99\%", xscale=:log10, ylim = [0,0.10])
savefig("error_distribution_EE_SO.png")
savefig("error_distribution_EE_SO.pdf")
plt

In [ ]:
plt = Plots.plot(sorted_residuals_EE_S4[1,:], xlabel=L"ℓ", ylabel=L"\frac{\left|\Delta C^{EE}\right|}{\sigma_{EE}}(\ell)\quad \mathrm{distribution}",legend=:topright, label = L"68\%")
Plots.plot!(plt, sorted_residuals_EE_S4[2,:], label = L"95\%")
Plots.plot!(plt, sorted_residuals_EE_S4[3,:], label = L"99\%", xscale=:log10)
savefig("error_distribution_EE.png")
plt

In [ ]:
plt = Plots.plot(sorted_residuals_EE_S4[1,:], xlabel=L"ℓ", ylabel=L"\frac{\left|\Delta C^{EE}\right|}{\sigma_{EE}}(\ell)\quad \mathrm{distribution}",legend=:topright, label = L"68\%")
Plots.plot!(plt, sorted_residuals_EE_S4[2,:], label = L"95\%")
Plots.plot!(plt, sorted_residuals_EE_S4[3,:], label = L"99\%", ylim=[0,0.1])
savefig("error_distribution_EE_S4.png")
savefig("error_distribution_EE_S4.pdf")
plt

# $C^{TE}(\ell)$ emulator test

In [ ]:
weights_TE = npzread(weights_folder*"weights_TE_lcdm.npy")
trained_emu_TE = Capse.SimpleChainsEmulator(Architecture= mlpd, Weights = weights_TE)
CℓTE_emu = Capse.CℓEmulator(TrainedEmulator = trained_emu_TE, ℓgrid = ℓ,
                             InMinMax = npzread(weights_folder*"inMinMax_lcdm.npy"),
                             OutMinMax = npzread(weights_folder*"outMinMaxCℓTE_lcdm.npy"));

In [ ]:
function compute_σTE(CℓTT, CℓEE, CℓTE, ℓ)
    result = Float64.(similar(ℓ))
    for i in 1:length(ℓ)
        result[i] = sqrt((CℓTT[i]*CℓEE[i]+(CℓTE[i])^2)/(2. *ℓ[i]+1.))
    end
    return result
end

function compute_σTE(CℓTT, CℓEE, CℓTE, NTT, NEE, ℓ, f_sky)
    result = Float64.(similar(ℓ))
    CℓTT .+= NTT
    CℓEE .+= NEE
    for i in 1:length(ℓ)
        result[i] = sqrt((CℓTT[i]*CℓEE[i]+(CℓTE[i])^2)/ (f_sky .* (2. *ℓ[i]+1.)))
    end
    return result
end

$$
\sigma_{\ell, \mathrm{CMB}}^{\mathrm{TE}} =\sqrt{\frac{1}{f_{\mathrm{sky}}(2 \ell+1)}} \times \sqrt{C_{\ell, \text { true }}^{\mathrm{TE}} C_{\ell, \text { true }}^{\mathrm{TE}}+\left(C_{\ell, \text { true }}^{\mathrm{TT}}+N_{\ell}^{\mathrm{TT}}\right)\left(C_{\ell, \text { true }}^{\mathrm{EE}}+N_{\ell}^{\mathrm{EE}}\right)},
$$
with $f_\mathrm{sky}=1$.

In [ ]:
CℓTE_capse = Capse.get_Cℓ(input_test, CℓTE_emu);

In [ ]:
plot_font = "Computer Modern"

Plots.default(titlefont = (16, plot_font), fontfamily=plot_font,
        linewidth=2, framestyle=:box, fg_legend =:black, label=nothing, grid=false, tickfontsize=12, size = (550, 400), labelfontsize = 13, dpi = 200)

plt = Plots.plot(xlabel=L"ℓ", ylabel=L"\frac{\ell(\ell+1)}{2\pi}C^{TE}(\ell)",legend=:bottomright)
Plots.plot!(plt, ℓ, CℓTE_test)# .* ℓ .*(ℓ .+1))
Plots.plot!(plt, ℓ, CℓTE_capse)# .* ℓ .*(ℓ .+1))

In [ ]:
σTE = compute_σTE(CℓTT_test, CℓEE_test, CℓTE_test, ℓ[3:5001]);

In [ ]:
plt = Plots.plot(xlabel=L"ℓ", ylabel=L"\frac{\left|\Delta C^{TE}\right|}{\sigma_{{TE}}}(\ell)",legend=:bottomright)
Plots.plot!(plt, ℓ[3:5001], abs.(CℓTE_test .- CℓTE_capse)./σTE )
savefig("residuals_ClTE.png")
plt

## Deep Test

In [ ]:
function evaluate_residuals_TE(Directory,  CℓTE_emu)
    my_values = zeros(20000, 4999)
    i = 0
    for (root, dirs, files) in walkdir(Directory)
        for file in files
            file_extension = file[findlast(isequal('.'),file):end]
            if file_extension == ".json"
                res = read_Cℓ_compute_res_TE(root, CℓTE_emu)
                i += 1
                my_values[i, :] = res
            end
        end
    end
    return my_values
end


function read_Cℓ_compute_res_TE(location, CℓTE_emu)
    json_string = read(location*"/cosmology_capse.json", String)
    cosmo_pars_test = JSON3.read(json_string)
    
    input_test = [cosmo_pars_test["ln10As"],
        cosmo_pars_test["ns"],
        cosmo_pars_test["H0"],
        cosmo_pars_test["ombh2"],
        cosmo_pars_test["omch2"],
        cosmo_pars_test["tau"]]
    
    CℓTT_test = npzread(location*"/clTT.npy")[3:5001]
    CℓEE_test = npzread(location*"/clEE.npy")[3:5001]
    CℓTE_test = npzread(location*"/clTE.npy")[3:5001]
    
    ℓ = npzread(location*"/ll.npy")[3:5001]
    
    CℓTE_capse = Capse.get_Cℓ(input_test, CℓTE_emu)
    σTE = compute_σTE(CℓTT_test, CℓEE_test, CℓTE_test, ℓ)
    
    res = abs.(CℓTE_test .- CℓTE_capse)./σTE
        
    return res
end

function evaluate_residuals_TE(Directory,  CℓTE_emu, NTT, NEE, f_sky)
    my_values = zeros(20000, 4997)
    i = 0
    for (root, dirs, files) in walkdir(Directory)
        for file in files
            file_extension = file[findlast(isequal('.'),file):end]
            if file_extension == ".json"
                res = read_Cℓ_compute_res_TE(root, CℓTE_emu, NTT, NEE, f_sky)
                i += 1
                my_values[i, :] = res
            end
        end
    end
    return my_values
end


function read_Cℓ_compute_res_TE(location, CℓTE_emu, NTT, NEE, f_sky)
    json_string = read(location*"/cosmology_capse.json", String)
    cosmo_pars_test = JSON3.read(json_string)
    
    input_test = [cosmo_pars_test["ln10As"],
        cosmo_pars_test["ns"],
        cosmo_pars_test["H0"],
        cosmo_pars_test["ombh2"],
        cosmo_pars_test["omch2"],
        cosmo_pars_test["tau"]]
    
    CℓTT_test = npzread(location*"/clTT.npy")[3:4999]
    CℓEE_test = npzread(location*"/clEE.npy")[3:4999]
    CℓTE_test = npzread(location*"/clTE.npy")[3:4999]
    
    ℓ = npzread(location*"/ll.npy")[3:4999]
    
    CℓTE_capse = Capse.get_Cℓ(input_test, CℓTE_emu)[1:4997]
    σTE = compute_σTE(CℓTT_test, CℓEE_test, CℓTE_test, NTT, NEE, ℓ, f_sky)
    
    res = abs.(CℓTE_test .- CℓTE_capse)./σTE
        
    return res
end

function evaluate_sorted_residuals_TE(Directory, CℓXX_emu, N_TT, N_EE, f_sky)
    residuals = evaluate_residuals_TE(Directory, CℓXX_emu, N_TT, N_EE, f_sky)
    n_elements, n_output = size(residuals)
    return sort_residuals(residuals, n_output, n_elements)
end

function evaluate_sorted_residuals_TE(Directory, CℓXX_emu)
    residuals = evaluate_residuals_TE(Directory, CℓXX_emu)
    n_elements, n_output = size(residuals)
    return sort_residuals(residuals, n_output, n_elements)
end

In [ ]:
sorted_residuals_TE = evaluate_sorted_residuals_TE(Cℓ_directory, CℓTE_emu)
sorted_residuals_TE_SO = evaluate_sorted_residuals_TE(Cℓ_directory, CℓTE_emu, TT_n_SO, EE_n_SO, 0.4)
sorted_residuals_TE_S4 = evaluate_sorted_residuals_TE(Cℓ_directory, CℓTE_emu, TT_n_S4, EE_n_S4, 0.4);

In [ ]:
npzwrite("sorted_residuals_TE.npy",    sorted_residuals_TE)
npzwrite("sorted_residuals_TE_SO.npy", sorted_residuals_TE_SO)
npzwrite("sorted_residuals_TE_S4.npy", sorted_residuals_TE_S4)

In [ ]:
plt = Plots.plot(sorted_residuals_TE[1,:],xlabel=L"ℓ", ylabel=L"\frac{\left|\Delta C^{TE}\right|}{\sigma_{{TE}}}(\ell)\quad \mathrm{distribution}",legend=:topright, label = L"68\%")
Plots.plot!(plt, sorted_residuals_TE[2,:], label = L"95\%")
Plots.plot!(plt, sorted_residuals_TE[3,:], label = L"99\%")
savefig("error_distribution_TE_CV.png")
savefig("error_distribution_TE_CV.pdf")
plt

In [ ]:
plt = Plots.plot(sorted_residuals_TE_SO[1,:],xlabel=L"ℓ", ylabel=L"\frac{\left|\Delta C^{TE}\right|}{\sigma_{{TE}}}(\ell)\quad \mathrm{distribution}",legend=:topright, label = L"68\%")
Plots.plot!(plt, sorted_residuals_TE_SO[2,:], label = L"95\%")
Plots.plot!(plt, sorted_residuals_TE_SO[3,:], label = L"99\%", xscale=:log10)
savefig("error_distribution_TE_SO.png")
savefig("error_distribution_TE_SO.pdf")
plt

In [ ]:
plt = Plots.plot(sorted_residuals_TE_S4[1,:],xlabel=L"ℓ", ylabel=L"\frac{\left|\Delta C^{TE}\right|}{\sigma_{{TE}}}(\ell)\quad \mathrm{distribution}",legend=:topright, label = L"68\%")
Plots.plot!(plt, sorted_residuals_TE_S4[2,:], label = L"95\%")
Plots.plot!(plt, sorted_residuals_TE_S4[3,:], label = L"99\%", xscale=:log10)
savefig("error_distribution_TE_S4.png")
savefig("error_distribution_TE_S4.pdf")
plt

# $C^{\phi\phi}(\ell)$ emulator test

In [ ]:
weights_PP = npzread(weights_folder*"weights_PP_lcdm.npy")
trained_emu_PP = Capse.SimpleChainsEmulator(Architecture= mlpd, Weights = weights_PP)
CℓPP_emu = Capse.CℓEmulator(TrainedEmulator = trained_emu_PP, ℓgrid = ℓ,
                             InMinMax = npzread(weights_folder*"inMinMax_lcdm.npy"),
                             OutMinMax = npzread(weights_folder*"outMinMaxCℓPP_lcdm.npy"));

In [ ]:
CℓPP_capse = Capse.get_Cℓ(input_test, CℓPP_emu);

In [ ]:
plot_font = "Computer Modern"

Plots.default(titlefont = (16, plot_font), fontfamily=plot_font,
        linewidth=2, framestyle=:box, fg_legend =:black, label=nothing, grid=false, tickfontsize=12, size = (550, 400), labelfontsize = 13, dpi = 200)

plt = Plots.plot(xlabel=L"ℓ", ylabel=L"\frac{(\ell(\ell+1))^2}{2\pi}C^{PP}(\ell)",legend=:bottomright)
Plots.plot!(plt, ℓ, CℓPP_test)
Plots.plot!(plt, ℓ, CℓPP_capse)

In [ ]:
σPP = compute_σXX(CℓPP_test[1:4999], ℓ[3:5001]);

In [ ]:
plt = Plots.plot(xlabel=L"ℓ", ylabel=L"\frac{\left|\Delta C^{PP}\right|}{\sigma_{C^{PP}}}(\ell)",legend=:bottomright)
Plots.plot!(plt, ℓ[3:5001], abs.(CℓPP_test .- CℓPP_capse)./σPP )
savefig("residuals_ClPP.png")
plt

## Deep Test

In [ ]:
sorted_residuals_PP = evaluate_sorted_residuals_XX(Cℓ_directory, CℓPP_emu, "PP")
sorted_residuals_PP_SO = evaluate_sorted_residuals_XX(Cℓ_directory, CℓPP_emu, "PP", PP_n_SO)
sorted_residuals_PP_S4 = evaluate_sorted_residuals_XX(Cℓ_directory, CℓPP_emu, "PP", PP_n_S4);

In [ ]:
npzwrite("sorted_residuals_PP.npy",    sorted_residuals_PP)
npzwrite("sorted_residuals_PP_SO.npy", sorted_residuals_PP_SO)
npzwrite("sorted_residuals_PP_S4.npy", sorted_residuals_PP_S4)

In [ ]:
plt = Plots.plot(sorted_residuals_PP[1,:], xlabel=L"ℓ", ylabel=L"\frac{\left|\Delta C^{PP}\right|}{\sigma_{PP}}(\ell)\quad \mathrm{distribution}",legend=:topright, label = L"68\%")
Plots.plot!(plt, sorted_residuals_PP[2,:], label = L"95\%")
Plots.plot!(plt, sorted_residuals_PP[3,:], label = L"99\%", xscale=:log10)
savefig("error_distribution_PP_CV.png")
savefig("error_distribution_PP_CV.pdf")
plt

In [ ]:
plt = Plots.plot(sorted_residuals_PP_SO[1,:], xlabel=L"ℓ", ylabel=L"\frac{\left|\Delta C^{PP}\right|}{\sigma_{PP}}(\ell)\quad \mathrm{distribution}",legend=:topright, label = L"68\%")
Plots.plot!(plt, sorted_residuals_PP_SO[2,:], label = L"95\%")
Plots.plot!(plt, sorted_residuals_PP_SO[3,:], label = L"99\%", xscale=:log10)
savefig("error_distribution_PP_SO.png")
savefig("error_distribution_PP_SO.pdf")
plt

In [ ]:
plt = Plots.plot(sorted_residuals_PP_S4[1,:], xlabel=L"ℓ", ylabel=L"\frac{\left|\Delta C^{PP}\right|}{\sigma_{PP}}(\ell)\quad \mathrm{distribution}",legend=:topright, label = L"68\%")
Plots.plot!(plt, sorted_residuals_PP_S4[2,:], label = L"95\%")
Plots.plot!(plt, sorted_residuals_PP_S4[3,:], label = L"99\%", xscale=:log10)
savefig("error_distribution_PP_S4.png")
savefig("error_distribution_PP_S4.pdf")
plt